In [8]:
from comet_ml import Experiment

import torch
import torch.nn as nn
import torchvision
import torchinfo

In [9]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_org = torchvision.models.resnet152(pretrained=True)
# model_org = model_org.to(device)

In [10]:
# データセットに合わせてモデルの出力次元を変更すするため，オリジナルのモデルの出力層への入力次元を取得

# model_org_features = model_org.classifier[6].in_features
# print(model_org_features)
# print(type(model_org_features))

In [11]:

# batch_size = 1
# torchinfo.summary(
#     model=model_org,
#     input_size=(batch_size, 3, 256, 256),
#     col_names=["input_size",
#                 "output_size"],
#     row_settings=["var_names"],
#     depth=3 
# )

In [12]:
class Args:
    def __init__(self):
        self.metadata_path = '/mnt/NAS-TVS872XT/dataset/'
        self.root = self.metadata_path
        self.annotation_path = self.metadata_path
        self.FRAMES_PER_CLIP = 16
        self.STEP_BETWEEN_CLIPS = 16
        self.BATCH_SIZE = 64
        self.NUM_WORKERS = 32  
        # self.CLIP_DURATION = 16 / 25
        self.CLIP_DURATION = (8 * 8) / 30  # (num_frames * sampling_rate)/fps
        self.VIDEO_NUM_SUBSAMPLED = 2  # 事前学習済みモデルに合わせて16→8
        self.UCF101_NUM_CLASSES = 101
        self.KINETIC400_NUM_CLASSES = 400
args = Args()

In [13]:
class Adapter(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(dim)
        self.conv1 = nn.Conv2d(dim, dim, 1)       
        self.bn2 = nn.BatchNorm2d(dim)


    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.conv1(out)
        
        out += residual
        out = self.bn2(out)        

        return out

In [14]:
class VideoToFrame(nn.Module):
    def __init__(self):
        super().__init__()

    def make_new_inputs(self, inputs):
        """
        動画データを画像データに分割

        Args:
            inputs (torch.Tensor): inputs
        Returns:
            new_inputs torch.Tensor: new_inputs
        """

        batch_size = inputs.size(0)
        num_frame = inputs.size(2)
        
        inputs = inputs.permute(0,2,1,3,4)
        outputs = inputs.reshape(batch_size*num_frame, 
                                inputs.size(2), 
                                inputs.size(3), 
                                inputs.size(4))

        return outputs

    def forward(self, x):
        x = self.make_new_inputs(x)
        return x

In [15]:
class FrameAvg(nn.Module):
    def __init__(self, batch_size, num_frame):
        super().__init__()
        self.batch_size = batch_size
        self.num_frame = num_frame


    def frame_out_to_video_out(self, input: torch.Tensor, batch_size, num_frame) -> torch.Tensor:
        """
        フレームごとの出力をビデオとしての出力に変換
        Args:
            input (torch.Tensor): フレームごとの出力
            batch_size (int): バッチサイズ
            num_frame (int): フレーム数

        Returns:
            torch.Tensor: ビデオとしての出力
        """
        input = input.reshape(self.batch_size, self.num_frame, -1)
        output = torch.mean(input, dim=1)
        return output

    def forward(self, x):
        x = self.frame_out_to_video_out(x, self.batch_size, self.num_frame)
        return x



In [16]:
class ReconstructNet(nn.Module):
    def __init__(self):
        super().__init__()
        model = torchvision.models.resnet152(pretrained=True)
        model_num_features = model.fc.in_features
        num_class = 101

        self.video_to_frame = VideoToFrame()
        self.net_bottom = nn.Sequential(
            model.conv1,
            model.bn1,
            model.relu,
            model.maxpool
        )
        
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avgpool = model.avgpool

        self.adapter = Adapter(512)

        self.net_top = nn.Sequential(
            nn.Linear(model_num_features, num_class)
        )

        self.frame_avg = FrameAvg(batch_size=args.BATCH_SIZE,
                                num_frame=args.VIDEO_NUM_SUBSAMPLED)
        
        # 学習させるパラメータ名
        self.update_param_names = ["adapter.bn1.weight", "adapter.bn1.bias",
                            "adapter.conv1.weight", "adapter.conv1.bias",
                            "adapter.bn2.weight", "adapter.bn2.bias", 
                            "net_top.0.weight", "net_top.0.bias"]
        # 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
        for name, param in self.named_parameters():
            if name in self.update_param_names:
                param.requires_grad = True
            else:
                param.requires_grad = False



    def forward(self, x):
        x = self.video_to_frame(x)
        x = self.net_bottom(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.adapter(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.net_top(x)
        x = self.frame_avg(x)
        return x

In [9]:
# model_new = ReconstructNet()
# model_new = model_new.to(device)

In [10]:
# batch_size = args.BATCH_SIZE*args.VIDEO_NUM_SUBSAMPLED

# torchinfo.summary(
#     model=model_new,
#     input_size=(batch_size, 3, 256, 256),
#     col_names=["input_size",
#                 "output_size"],
#     row_settings=["var_names"],
#     depth=3 
# )

In [11]:
# # ダミーデータを用意し，出力が一致するか確認
# data = torch.randn(1, 3, 256, 256).to(device)
# print(data.shape)
# print(type(data))

# # data1 = torch.full((1,3,256,256), 2).to(device)
# # print(data1.shape)
# # print(type(data1))

In [12]:
# model_org.eval()
# model_new.eval()
# output_org = model_org(data).max(axis=1)
# output_new = model_new(data).max(axis=1)
# print(output_org)
# print(output_new)
# output_org = model_org(data)
# output_new = model_new(data)
# # print(output_org.shape)
# print(output_new.shape)


In [13]:
# テンソルの出力のまま比較する場合
# flag = torch.allclose(output_org,output_new, atol=1e-8)
# print(flag)

### 実際に学習させてみる
- data：Kinetics400
- model:vgg16 (Imagenetでpretrain)

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DistributedSampler, RandomSampler

from torchvision import transforms

from pytorchvideo.models import x3d
from pytorchvideo.data import Ucf101, RandomClipSampler, UniformClipSampler, Kinetics

# from torchvision.transforms._transforms_video import (
#     CenterCropVideo,
#     NormalizeVideo,
# )

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)

from tqdm.notebook import tqdm
from collections import OrderedDict
import itertools
import os

from comet_ml import Experiment

In [19]:
class LimitDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos



In [20]:
def get_ucf101(subset):
    """
    ucf101のデータセットを取得

    Args:
        subset (str): "train" or "test"

    Returns:
        pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset: 取得したデータセット
    """
    subset_root_Ucf101 = 'ucfTrainTestlist/trainlist01.txt'
    if subset == "test":
        subset_root_Ucf101 = 'ucfTrainTestlist/testlist.txt'

    args = Args()
    transform = Compose([
        ApplyTransformToKey(
            key="video",
            transform=Compose([
                UniformTemporalSubsample(args.VIDEO_NUM_SUBSAMPLED),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                RandomShortSideScale(min_size=256, max_size=320,),
                RandomCrop(224),
                RandomHorizontalFlip(),
            ]),
        ),
        ApplyTransformToKey(
            key="label",
            transform=transforms.Lambda(lambda x: x-1),
        ),
        RemoveKey("audio"),
    ])

    root_ucf101 = '/mnt/dataset/UCF101/'

    dataset = Ucf101(
        data_path=root_ucf101 + subset_root_Ucf101,
        video_path_prefix=root_ucf101 + 'video/',
        clip_sampler=RandomClipSampler(clip_duration=args.CLIP_DURATION),
        video_sampler=RandomSampler,
        decode_audio=False,
        transform=transform,
    )

    return dataset

In [21]:

def get_kinetics(subset):
    """
    Kinetics400のデータセットを取得

    Args:
        subset (str): "train" or "val" or "test"

    Returns:
        pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset: 取得したデータセット
    """
    args = Args()
    transform = Compose([
        ApplyTransformToKey(
            key="video",
            transform=Compose([
                UniformTemporalSubsample(args.VIDEO_NUM_SUBSAMPLED),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ShortSideScale(size=256),
                # RandomShortSideScale(min_size=256, max_size=320,),
                # CenterCropVideo(crop_size=(256, 256)),
                CenterCrop(256),
                # RandomCrop(224),
                RandomHorizontalFlip(),
            ]),
        ),
        ApplyTransformToKey(
            key="label",
            transform=transforms.Lambda(lambda x: x),
        ),
        RemoveKey("audio"),
    ])

    root_kinetics = '/mnt/NAS-TVS872XT/dataset/Kinetics400/'

    if subset == "test":
        dataset = Kinetics(
            data_path=root_kinetics + "test_list.txt",
            video_path_prefix=root_kinetics + 'test/',
            clip_sampler=RandomClipSampler(clip_duration=args.CLIP_DURATION),
            video_sampler=RandomSampler,
            decode_audio=False,
            transform=transform,
        )
        return dataset
    else:
        dataset = Kinetics(
            data_path=root_kinetics + subset,
            video_path_prefix=root_kinetics + subset,
            clip_sampler=RandomClipSampler(clip_duration=args.CLIP_DURATION),
            video_sampler=RandomSampler,
            decode_audio=False,
            transform=transform,
        )
        return dataset

    return False

In [22]:
def make_loader(dataset):
    """
    データローダーを作成

    Args:
        dataset (pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset): get_datasetメソッドで取得したdataset

    Returns:
        torch.utils.data.DataLoader: 取得したデータローダー
    """
    args = Args()
    loader = DataLoader(LimitDataset(dataset),
                        batch_size=args.BATCH_SIZE,
                        drop_last=True,
                        num_workers=args.NUM_WORKERS)
    return loader

In [23]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    https://github.com/machine-perception-robotics-group/attention_branch_network/blob/ced1d97303792ac6d56442571d71bb0572b3efd8/utils/misc.py#L59
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if type(val) == torch.Tensor:
            val = val.item()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def top1(outputs, targets):
    batch_size = outputs.size(0)
    _, predicted = outputs.max(1)
    return predicted.eq(targets).sum().item() / batch_size

In [24]:
# def make_new_inputs(inputs):
#     """
#     動画データを画像データに分割

#     Args:
#         inputs (torch.Tensor): inputs
#     Returns:
#         new_inputs torch.Tensor: new_inputs
#     """

#     batch_size = inputs.size(0)
#     num_frame = inputs.size(2)
    
#     inputs = inputs.permute(0,2,1,3,4)
#     outputs = inputs.reshape(batch_size*num_frame, 
#                              inputs.size(2), 
#                              inputs.size(3), 
#                              inputs.size(4))

#     return outputs

In [25]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
dataset = get_ucf101("train")
# dataset.video_sampler._num_samples = 100
train_loader = make_loader(dataset)

cuda:2


### torchinfoのNon-trainable params
- torchinfoで表示させる前に表示させいたいモデルをGPUに送るとNon-trainable paramsが正しく表示されない
- またtorchinfoで表示させる際にデフォルトでcuda:0にモデルが送られるため，cuda:0以外にモデルを送っているとうまくいかない（torchinfoで実際にダミーデータをモデルに入れて出力のサイズとかを測っているからモデルをGPUに送っていてそれがデフォルトでcuda:0なのかも）

In [26]:
model = ReconstructNet()
model = model.to(device)
# print(device)

# if device == "cuda":
#     print("test")
#     model = torch.nn.DataParallel(model)
#     cudnn.benchmark = True
#     print("test")

# model = torch.nn.parallel.DistributedDataParallel(model)
# model = torch.nn.DataParallel(model)
# torch.backends.cudnn.benchmark = True


In [27]:
size = args.BATCH_SIZE*args.VIDEO_NUM_SUBSAMPLED

torchinfo.summary(
    model=model,
    input_size=(args.BATCH_SIZE, 3, args.VIDEO_NUM_SUBSAMPLED, 256, 256),
    col_names=["input_size",
                "output_size"],
    row_settings=["var_names"],
    depth=2 
)

Layer (type (var_name))                  Input Shape               Output Shape
ReconstructNet                           --                        --
├─VideoToFrame (video_to_frame)          [64, 3, 2, 256, 256]      [128, 3, 256, 256]
├─Sequential (net_bottom)                [128, 3, 256, 256]        [128, 64, 64, 64]
│    └─Conv2d (0)                        [128, 3, 256, 256]        [128, 64, 128, 128]
│    └─BatchNorm2d (1)                   [128, 64, 128, 128]       [128, 64, 128, 128]
│    └─ReLU (2)                          [128, 64, 128, 128]       [128, 64, 128, 128]
│    └─MaxPool2d (3)                     [128, 64, 128, 128]       [128, 64, 64, 64]
├─Sequential (layer1)                    [128, 64, 64, 64]         [128, 256, 64, 64]
│    └─Bottleneck (0)                    [128, 64, 64, 64]         [128, 256, 64, 64]
│    └─Bottleneck (1)                    [128, 256, 64, 64]        [128, 256, 64, 64]
│    └─Bottleneck (2)                    [128, 256, 64, 64]        [128, 25

In [24]:
# torch.backends.cudnn.version()

In [25]:
# パラメータの名前を確認する
# for name, params in model.named_parameters():
#     print(name)

In [26]:
# # # 転移学習で学習させるパラメータを、変数params_to_updateに格納する
# # # optimizerの引数にparams_to_updataを入れることもできる
# params_to_update = []

# # 学習させるパラメータ名
# update_param_names = ["adapter.bn1.weight", "adapter.bn1.bias",
#                       "adapter.conv1.weight", "adapter.conv1.bias",
#                       "adapter.bn2.weight", "adapter.bn2.bias", 
#                       "net_top.0.weight", "net_top.0.bias"]

# # 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
# for name, param in model.named_parameters():
#     if name in update_param_names:
#         param.requires_grad = True
#         params_to_update.append(param)
#         print(name)
#     else:
#         param.requires_grad = False

In [27]:
# for name, param in model.named_parameters():
#     param.requires_grad = True
#     print(type(param))

In [ ]:
hyper_params = {
    "Dataset": "UCF101",
    "batch_size": args.BATCH_SIZE,
    "num_frame:":args.VIDEO_NUM_SUBSAMPLED,
}

experiment = Experiment(
    api_key="TawRAwNJiQjPaSMvBAwk4L4pF",
    project_name="feeature-extract",
    workspace="kazukiomi",
)

experiment.add_tag('pytorch')
experiment.log_parameters(hyper_params)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/kazukiomi/feeature-extract/29f848e426384e5fa2eeeaeffe28cc42



In [28]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [29]:
num_epochs = 4

acc_list = []
loss_list = []
step = 0

with tqdm(range(num_epochs)) as pbar_epoch:
    for epoch in pbar_epoch:
        pbar_epoch.set_description("[Epoch %d]" % (epoch))


        with tqdm(enumerate(train_loader),
                  total=len(train_loader),
                  leave=True) as pbar_batch:

            train_loss = AverageMeter()
            train_acc = AverageMeter()
            model.train()


            for batch_idx, batch in pbar_batch:
                pbar_batch.set_description("[Epoch :{}]".format(epoch))

                inputs = batch['video'].to(device)
                labels = batch['label'].to(device)

                new_inputs = make_new_inputs(inputs)
                new_inputs = new_inputs.to(device)
                
                bs = inputs.size(0)
                new_bs = new_inputs.size(0)  # current batch size, may vary at the end of the epoch

                optimizer.zero_grad()
                outputs = model(new_inputs)
                # print(outputs.device)
                # print(new_labels.device)

                # ここでフレームごとの出力をビデオごとの出力に変換する
                # video_outputs = frame_out_to_video_out(outputs, bs, args.VIDEO_NUM_SUBSAMPLED) 

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                
                preds = torch.squeeze(outputs.max(dim=1)[1])
                # print(video_outputs.shape)
                # print(preds.shape)

                # acc = (preds == labels).float().mean().item()
                # acc_list.append(acc)
                # pbar_batch.set_postfix(OrderedDict(loss=loss.item(),acc=acc))

                train_loss.update(loss, bs)
                train_acc.update(top1(outputs, labels), bs)

                pbar_batch.set_postfix_str(
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ''.format(
                    train_loss.avg, train_acc.avg,
                    train_loss.val, train_acc.val,
                ))

                # experiment.log_metric("accuracy", train_acc.val, step=step)
                # step += 1

            acc_list.append(train_acc.avg)
            loss_list.append(train_loss.avg)
        pbar_epoch.set_postfix(OrderedDict(
            acc=sum(acc_list)/len(acc_list),
            loss=sum(loss_list)/len(loss_list)
        ))



RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 2; 10.92 GiB total capacity; 9.89 GiB already allocated; 76.69 MiB free; 10.07 GiB reserved in total by PyTorch)

In [31]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/kazukiomi/feeature-extract/cf63b0cbf43c41ad885bdbb745318859
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [1192] : (0.0, 0.9375)
COMET INFO:     loss [120]      : (0.19330792129039764, 13.611698150634766)
COMET INFO:   Parameters:
COMET INFO:     Dataset    : UCF101
COMET INFO:     batch_size : 32
COMET INFO:     num_frame: : 2
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (27.87 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     model graph              : 1
COMET INFO:     notebook                 : 1
COMET INFO:     os packages              : 1
COMET INFO: 